# Instance #0

This instance used classic Le-Net5 model. 

## Section 1 Define Super Training Parameters

- epoch: define the iteration of the train
- batch_size: define the train batch size. It depends on how large the memory is. CIFAR-10 is a very small images. 200 - 500 should be good. 
- test_size: define the test batch size.  
- learn rate (lr): The start learn rate for Agagrad
- keep_prob: the probability of the training parameter
- augument: To have a better training effect, the image augument is always True

In [1]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Sun Apr  7 22:08:46 2019

@author: Deechean
"""
from cifar10 import cifar10
import os
import tensorflow as tf
from lenet5 import LeNet5
import tf_general as tfg
import numpy as np
from train_log import train_log

FLAGS = tf.flags.FLAGS

try:
    #Super parameter definition
    tf.flags.DEFINE_string('f', '', 'kernel')
    tf.flags.DEFINE_integer('epoch', 40000, 'epoch')
    tf.flags.DEFINE_integer('batch_size',200, 'batch size')
    tf.flags.DEFINE_integer('test_size', 200, 'test size')
    tf.flags.DEFINE_float('lr', 0.01, 'learning rate')
    tf.flags.DEFINE_float('keep_prob', 0.5, 'keep prob for drop lay')
    tf.flags.DEFINE_boolean('augument', True, 'if image augument is applied') 
    #Other training parameter                        
    tf.flags.DEFINE_float('ckpt_frequency', 250, 'frequency to save checkpoint')
    tf.flags.DEFINE_boolean('restore', False, 'restore from checkpoint and run test')
    print('parameters were defined.')
except:
    print('parameters have been defined.')

print("learn Rate =",FLAGS.lr, "keep Prob=", FLAGS.keep_prob, " image augument=",FLAGS.augument)   
CONTINUE = 0
RUN = 1

parameters were defined.
learn Rate = 0.01 keep Prob= 0.5  image augument= True


## Generate Checkpoint dir and Log dir

- Checkpoint dir is saved in variable **../Le-Net5-Log/Le-Net5_CLASS/ckpt_RUN**, if the dir doesn't exist then create it. 
- Log file dir is saved in variable **../Le-Net5-Log/Le-Net5_CLASS/log_RUN**, if the dir doesn't exist then create it. 
- data_path is the position of the CIFAR-10 image data
The reason to save the model and log outside the project is to avoid effect the git code management

In [2]:
ckpt_dir = '../Le-Net5-Log/Le-Net5_CLASS/ckpt_'+str(RUN)+'/'
if not os.path.exists(ckpt_dir):
    os.makedirs(ckpt_dir)

log_dir = '../Le-Net5-Log/Le-Net5_CLASS/log_'+str(RUN)+'/'
log = train_log(log_dir)
if not os.path.exists(log_dir):
    os.makedirs(log_dir)

data_path = '../cifar-10-batches-py/'
if not os.path.exists(data_path):
    print('The data path doesn\'t exist. Please check if it is a correct data path.')
    

## Input layer 
Use the data feeder to provide the training data. Therefore we define a placeholder with the same structure of the input data. 
The CIFAR-10 data is 60000 RGB images with each size is 32x32. The data structure should be[batchsize, 32,32,3]. Input channels are 3.  

In [3]:
with tf.device('/cpu:0'):
    with tf.name_scope('input'):
        x = tf.placeholder(tf.float32, [None,32,32,3], name='x_input')
        keep_prob = tf.placeholder(tf.float32, name='keep_prob')
        y_ = tf.placeholder(tf.int64, [None], name='labels')

In [4]:
with tf.device('/cpu:1'):
    with tf.name_scope('prediction'):
        le_net5 = LeNet5(x, keep_prob)
        y = le_net5.prediction
        """training parameters log
        x_norm = le_net5.x_norm
        conv1 = le_net5.conv1
        pool1 = le_net5.pool1
        conv2 = le_net5.conv2
        pool2 = le_net5.pool2
        """

Instructions for updating:
Colocations handled automatically by placer.
conv_1:  (?, 28, 28, 6)
pool_1:  (?, 14, 14, 6)
conv_2:  (?, 10, 10, 16)
pool_2:  (?, 5, 5, 16)
flat_1: (?, 400)
fc_1:  (?, 120)
fc_2:  (?, 84)
fc_3:  (?, 10)
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
drop_out:  (?, 10)
prediction:  (?, 10)


## Calculate the cross entropy as the loss

In [5]:
with tf.device('/cpu:2'):
    with tf.name_scope('cross_entropy'):
        cross_entropy = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits=y,
                                                                                      labels=y_, 
                                                                                      name="cross_entropy_per_example"))

## Use Adagrad to minimize the loss

In [6]:
with tf.device('/cpu:3'):
    with tf.name_scope('train_step'):
        train_step = tf.train.AdagradOptimizer(FLAGS.lr).minimize(cross_entropy)

## Calculate the reduce mean as the accuracy

In [7]:
with tf.device('/cpu:0'):
    prediction =tf.argmax(y, 1)
    accuracy = tf.reduce_mean(tf.cast( tf.equal(prediction,y_), tf.float32))

## Now, let's start the training...

In [8]:
import time

if __name__ == '__main__':
    data = cifar10(data_path);

    with tf.Session(config=tf.ConfigProto(
                            device_count={"CPU":4},
                            inter_op_parallelism_threads=1,
                            intra_op_parallelism_threads=1)) as sess:
        sess.run(tf.global_variables_initializer())
        saver = tf.train.Saver(max_to_keep=1)
        if CONTINUE != 0:
            model_file=tf.train.latest_checkpoint(ckpt_dir)
            saver.restore(sess,model_file)
        for i in range(CONTINUE, FLAGS.epoch):
            train_image, train_label,_ = data.get_train_batch(FLAGS.batch_size,FLAGS.augument)
            loss, _,accuracy_rate = sess.run([cross_entropy, train_step, accuracy], 
                                             feed_dict={keep_prob: FLAGS.keep_prob, x:train_image, y_:train_label})
            
            log.add_log('train_accuracy',i, accuracy_rate)
            log.add_log('train_loss',i, loss)
            if (i+1) % FLAGS.ckpt_frequency == 0:  #保存预测模型
                saver.save(sess,ckpt_dir+'cifar10_'+str(i+1)+'.ckpt',global_step=i+1)
                acc_accuracy = 0
                for j in range(int(10000/FLAGS.test_size)):                    
                    test_image, test_label,test_index = data.get_test_batch(FLAGS.test_size)
                    accuracy_rate, output = sess.run([accuracy,prediction],
                                                     feed_dict={keep_prob: 1, x:test_image, y_:test_label})
                    acc_accuracy += accuracy_rate
                    log.add_log('test_index',i, test_index)
                    log.add_log('output',i, output)
                accuracy_rate = acc_accuracy/10000*FLAGS.test_size
                print(time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()) + 
                      ' iter ' + str(i) + ', Test accuracy:' +str(round(accuracy_rate*100,2))+'%')
                log.add_log('test_accuracy',i, accuracy_rate)
                log.SaveToFile()
    tf.reset_default_graph()

train set length: 50000
test set length: 10000
2019-06-11 22:12:47 iter 249, Test accuracy:17.36%
Instructions for updating:
Use standard file APIs to delete files with this prefix.
2019-06-11 22:15:39 iter 499, Test accuracy:17.62%
2019-06-11 22:18:31 iter 749, Test accuracy:18.53%
2019-06-11 22:22:03 iter 999, Test accuracy:19.35%
2019-06-11 22:25:13 iter 1249, Test accuracy:19.57%
2019-06-11 22:28:14 iter 1499, Test accuracy:20.22%
2019-06-11 22:31:36 iter 1749, Test accuracy:20.24%


KeyboardInterrupt: 